# Imports

In [1]:
import pandas as pd
import numpy as np

# Data Exploration

In [2]:
path_base_01 = r'C:\Projetos\case-ami-saude\data\raw\Base 1 – Internações Hospitalares.csv'
path_base_02 = r'C:\Projetos\case-ami-saude\data\raw\Base 2 – Itens da Internação.csv'
df_base_01 = pd.read_csv(path_base_01)
df_base_02 = pd.read_csv(path_base_02)

In [4]:
df_base_01

,senha_internacao,beneficiario_id,numero_carteirinha,nome_beneficiario,data_nascimento,idade,sexo,uf,municipio,hospital_id,...,uti_flag,suporte_ventilatorio_flag,hemodialise_flag,tempo_autorizacao_horas,auditoria_responsavel,empresa_auditoria,status_regulacao,glosa_flag,valor_total_conta,valor_pago
0,SI20250000001,SI20250000001,CAR405957,P.M.M.,1992-06-17 00:00:00,33,M,SP,Santos,H070,...,Não,Não,Não,30.0,Terceirizada,Empresa Terceira 3,Autorizada,Não,112005.19,112005.19
1,SI20250000002,SI20250000002,CAR658953,V.S.M.,1987-10-24 00:00:00,38,F,MS,Dourados,H013,...,Sim,Não,Sim,22.0,Terceirizada,Empresa Terceira 3,Autorizada com ressalva,Sim,357271.95,303110.50
2,SI20250000003,SI20250000003,CAR547463,N.M.A.,1929-02-16 00:00:00,96,F,MA,Imperatriz,H001,...,Não,Não,Não,26.0,Terceirizada,Empresa Terceira 2,Pendente,Não,32603.64,32603.64
3,SI20250000004,SI20250000004,CAR181581,P.C.N.,1976-10-29 00:00:00,49,M,MG,Juiz de Fora,H043,...,Não,Não,Não,34.0,Terceirizada,Empresa Terceira 2,Autorizada,Sim,24660.48,23423.61
4,SI20250000005,SI20250000005,CAR677468,B.A.R.,1981-02-07 00:00:00,45,M,PA,Belém,H028,...,Não,Não,Sim,21.0,Terceirizada,Empresa Terceira 3,Autorizada,Sim,46771.29,40277.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,SI20250019996,SI20250019996,CAR811091,P.N.C.,1983-05-08 00:00:00,42,F,SP,Santos,H043,...,Não,Não,Não,29.0,Terceirizada,Empresa Terceira 2,Pendente,Não,18658.17,18658.17
19996,SI20250019997,SI20250019997,CAR148487,L.M.T.,1952-11-05 00:00:00,73,M,MA,Imperatriz,H008,...,Não,Sim,Não,27.0,Terceirizada,Empresa Terceira 1,Autorizada,Não,30272.52,30272.52
19997,SI20250019998,SI20250019998,CAR540753,M.F.M.,1995-06-28 00:00:00,30,F,PR,Curitiba,H061,...,Não,Não,Não,13.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,22906.91,22906.91
19998,SI20250019999,SI20250019999,CAR367435,A.M.O.,1948-11-02 00:00:00,77,F,MG,Contagem,H007,...,Não,Não,Não,40.0,Terceirizada,Empresa Terceira 2,Autorizada,Não,12890.38,12890.38


In [5]:
df_base_02

,item_id,senha_internacao,data_item,tipo_item,subtipo_item,codigo_item,descricao_item,quantidade_solicitada,quantidade_autorizada,unidade_medida,valor_unitario,valor_total_item,setor_execucao,flag_pacote,glosa_item_flag,motivo_glosa,valor_glosado
0,IT000047081,SI20250002374,2025-04-23 16:36:36,Exame,Exames de imagem (un),EX_IMG,Exames de imagem (un),2,2,un,1675.92,3351.84,SADT,Não,Sim,Administrativa - código incompatível,3351.84
1,IT000290746,SI20250014538,2024-08-17 05:07:19,Taxa,Honorários (un),TX_HON,Honorários (un),3,3,un,1158.60,3475.80,Administrativo,Não,Sim,Técnica - item não pertinente ao CID,3475.80
2,IT000341194,SI20250017099,2024-10-12 09:09:58,Procedimento,Procedimento Cirúrgico,PR_CIR,Procedimento Cirúrgico,1,1,un,4314.57,4314.57,Centro Cirúrgico,Não,Não,NaN,0.00
3,IT000179585,SI20250008978,2025-10-23 17:20:13,Diária,Diária Enfermaria,DI_ENF,Diária Enfermaria,8,8,diária,559.87,4478.96,Enfermaria,Não,Não,NaN,0.00
4,IT000367705,SI20250018405,2024-07-31 13:28:24,Medicamento,Medicamentos (dose),MD_GER,Medicamentos (dose),4,4,dose,89.72,358.88,Farmácia,Não,Não,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,IT000401849,SI20250011218,2025-06-23 13:45:13,Diária,Diária Enfermaria,DI_ENF,Diária Enfermaria,17,17,diária,1369.52,23281.77,Enfermaria,Não,Não,NaN,0.00
399996,IT000401850,SI20250004187,2025-10-11 17:31:34,Medicamento,Antibiótico (dose),MD_ATB,Antibiótico (dose),2,2,dose,62.85,125.70,Farmácia,Não,Não,NaN,0.00
399997,IT000401851,SI20250011297,2025-06-20 20:24:33,Taxa,Taxas e materiais (un),TX_MAT,Taxas e materiais (un),1,0,un,210.93,0.00,Enfermaria,Não,Não,NaN,0.00
399998,IT000401852,SI20250009120,2025-11-21 00:11:00,Terapia seriada,Fonoaudiologia (sessão),TS_FON,Fonoaudiologia (sessão),4,4,sessão,84.44,337.76,Enfermaria,Não,Não,NaN,0.00


In [6]:
df_base_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   senha_internacao              20000 non-null  object 
 1   beneficiario_id               20000 non-null  object 
 2   numero_carteirinha            20000 non-null  object 
 3   nome_beneficiario             20000 non-null  object 
 4   data_nascimento               20000 non-null  object 
 5   idade                         20000 non-null  int64  
 6   sexo                          20000 non-null  object 
 7   uf                            20000 non-null  object 
 8   municipio                     20000 non-null  object 
 9   hospital_id                   20000 non-null  object 
 10  hospital_nome                 20000 non-null  object 
 11  perfil_hospital               20000 non-null  object 
 12  tipo_plano                    20000 non-null  object 
 13  s

In [7]:
df_base_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   item_id                400000 non-null  object 
 1   senha_internacao       400000 non-null  object 
 2   data_item              400000 non-null  object 
 3   tipo_item              400000 non-null  object 
 4   subtipo_item           400000 non-null  object 
 5   codigo_item            400000 non-null  object 
 6   descricao_item         400000 non-null  object 
 7   quantidade_solicitada  400000 non-null  int64  
 8   quantidade_autorizada  400000 non-null  int64  
 9   unidade_medida         400000 non-null  object 
 10  valor_unitario         400000 non-null  float64
 11  valor_total_item       400000 non-null  float64
 12  setor_execucao         400000 non-null  object 
 13  flag_pacote            400000 non-null  object 
 14  glosa_item_flag        400000 non-nu

In [8]:
df_base_01.columns

Index(['senha_internacao', 'beneficiario_id', 'numero_carteirinha',
       'nome_beneficiario', 'data_nascimento', 'idade', 'sexo', 'uf',
       'municipio', 'hospital_id', 'hospital_nome', 'perfil_hospital',
       'tipo_plano', 'segmentacao_plano', 'acomodacao',
       'data_solicitacao_autorizacao', 'data_autorizacao_senha',
       'data_admissao', 'data_alta', 'motivo_alta', 'carater_internacao',
       'tipo_internacao', 'especialidade_responsavel', 'cid_principal',
       'cid_secundario', 'complexidade', 'uti_flag',
       'suporte_ventilatorio_flag', 'hemodialise_flag',
       'tempo_autorizacao_horas', 'auditoria_responsavel', 'empresa_auditoria',
       'status_regulacao', 'glosa_flag', 'valor_total_conta', 'valor_pago'],
      dtype='object')

In [11]:
df_base_02.columns

Index(['item_id', 'senha_internacao', 'data_item', 'tipo_item', 'subtipo_item',
       'codigo_item', 'descricao_item', 'quantidade_solicitada',
       'quantidade_autorizada', 'unidade_medida', 'valor_unitario',
       'valor_total_item', 'setor_execucao', 'flag_pacote', 'glosa_item_flag',
       'motivo_glosa', 'valor_glosado'],
      dtype='object')